In [ ]:
import mapsims
import numpy as np
import healpy as hp
import pysm3.units as u
from pathlib import Path
%matplotlib inline
import matplotlib.pyplot as plt

# Access CMB realizations

Check the documentation of the [`201911_lensed_cmb`](https://github.com/simonsobs/map_based_simulations/tree/master/201911_lensed_cmb) release.

This release contains 100 lensed CMB maps with no primordial tensor modes.

In [ ]:
channel = mapsims.parse_channels("ST0_UHF1")[0]

# Load maps from a map based release at NERSC

In [ ]:
folder = "/project/projectdirs/sobs/v4_sims/mbs/201911_lensed_cmb"

In [ ]:
!ls $folder/512

In [ ]:
filename_template = "{nside}/{content}/{num:04d}/simonsobs_{content}_uKCMB_{telescope}{band}_nside{nside}_{num:04d}.fits"

In [ ]:
filenames = [Path(folder) / filename_template.format(nside=512, content="cmb", num=num, 
                        telescope=channel.telescope.lower(), band=channel.band) for num in range(10)]

In [ ]:
maps = [hp.ma(hp.read_map(filename, (0,1,2))) for filename in filenames]

# Apply a scanning strategy cut

The `mapsims.SONoiseSimulator` class, which is used to simulate noise, can also automatically download hitmaps that were simulated by TOAST in time domain.

In [ ]:
noise = mapsims.SONoiseSimulator(nside=512)

In [ ]:
hitmaps, ave_nhits = noise.get_hitmaps(tube=channel.tube)

For this simulation set the hitmaps are the same for both channels but in principle they might be different.

In [ ]:
for each in maps:
    each[:, hitmaps[0] == 0] = hp.UNSEEN

In [ ]:
hp.mollview(maps[0][1])

In [ ]:
cls = [hp.anafast(each, lmax=512) for each in maps]

In [ ]:
plt.figure()
plt.title("EE Power spectrum")
for cl in cls:
    plt.loglog(cl[1])
plt.xlabel("ell")

# Add foregrounds

We can load concurrently some maps from the foregrounds release and co-add them,
we can get the paths from the previous notebook.

In [ ]:
folder = "/project/projectdirs/sobs/v4_sims/mbs/201906_highres_foregrounds_extragalactic_tophat"

In [ ]:
filename_template = "{nside}/{content}/{num:04d}/simonsobs_{content}_uKCMB_{telescope}{band}_nside{nside}_{num:04d}.fits"

In [ ]:
filename = Path(folder) / filename_template.format(nside=512, content="dust", num=0, 
                                                   telescope=channel.telescope.lower(), band=channel.band)

In [ ]:
dust = hp.read_map(filename, (0,1,2))

Need to be careful with this operation, we do not want to override UNSEEN pixels,
here it works because we are using `hp.ma` masked array which handles that automatically.

In [ ]:
for each in maps:
    each += dust

In [ ]:
hp.mollview(maps[0][1])